### Defining   an      "adopted   user"      as   a   user   who         has   logged   into   the   product   on   three   separate  days   in   at   least   one   seven day   period,   identify   which   factors   predict   future   user  adoption. 

+ We   suggest   spending   1-2   hours   on   this,   but   you're   welcome   to   spend   more   or   less.  Please   send   us   a   brief   writeup   of   your   findings   (the   more   concise,   the   better,    no   more  than   one   page),   along   with   any   summary   tables,   graphs,   code,   or   queries   that   can   help  us   understand   your   approach.   Please   note   any   factors   you   considered   or   investigation  you   did,   even   if   they   did   not   pan   out.   Feel   free   to   identify   any   further   research   or   data  you   think   would   be   valuable.  

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Load in engagement data

engage_data = pd.read_csv("C:/Users/vsingh/OneDrive - Red Clay Consulting, Inc/Desktop/takehome_user_engagement.csv")

In [31]:
#Load in user data
user_data = pd.read_csv("C:/Users/vsingh/OneDrive - Red Clay Consulting, Inc/Desktop/takehome_users.csv", encoding='latin-1')

In [32]:
engage_data.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [33]:
user_data.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [37]:
#Create new column
engage_data["datetime"] = pd.to_datetime(engage_data.time_stamp)
engage_data_counts = engage_data["user_id"].value_counts()
engage_data_3plus = engage_data[engage_data["user_id"].isin(engage_data_counts[engage_data_counts > 2].index)]
engage_data_3plus.head()

,time_stamp,user_id,visited,datetime
1,2013-11-15 03:45:04,2,1,2013-11-15 03:45:04
2,2013-11-29 03:45:04,2,1,2013-11-29 03:45:04
3,2013-12-09 03:45:04,2,1,2013-12-09 03:45:04
4,2013-12-25 03:45:04,2,1,2013-12-25 03:45:04
5,2013-12-31 03:45:04,2,1,2013-12-31 03:45:04


In [41]:
#2248 Unique users
len(engage_data_3plus['user_id'].unique())

2248

In [59]:
#Retrieving adopted users
adopted_users = []
for user in engage_data_3plus["user_id"].unique():
    user_engagement = engage_data_3plus[engage_data_3plus["user_id"] == user]
    user_datetime = user_engagement["datetime"].reset_index()["datetime"]
    for i in range(len(user_datetime) - 2):
        time_interval = user_datetime[i + 2] - user_datetime[i]
        if time_interval < pd.Timedelta("7 days"):
            adopted_users.append(user)
            break

len(adopted_users)

1602

In [58]:
print(len(adopted_users)/len(engage_data_3plus['user_id'].unique())*100, "% of users are adopted.")

71.26334519572953 % of users are adopted.


In [48]:
user_adoption = user_data["object_id"].isin(adopted_users)
pd.crosstab(user_adoption, user_data["creation_source"], normalize = "columns")

creation_source,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
object_id,,,,,
False,0.833564,0.870005,0.922312,0.859607,0.832491
True,0.166436,0.129995,0.077688,0.140393,0.167509


In [51]:
pd.crosstab(user_adoption, user_data["opted_in_to_mailing_list"], normalize = "columns")

opted_in_to_mailing_list,0,1
object_id,,
False,0.868088,0.861723
True,0.131912,0.138277


In [53]:
user_data["org_id"].value_counts().head(20)

0     319
1     233
2     201
3     168
4     159
6     138
5     128
9     124
7     119
10    104
8      97
14     87
11     75
12     75
17     74
18     73
13     72
16     72
20     68
15     64
Name: org_id, dtype: int64

In [54]:
pd.to_datetime(user_data["creation_time"]).apply(lambda x: x.year).value_counts()

2013    5676
2014    3527
2012    2797
Name: creation_time, dtype: int64

In [56]:
adopted_months = pd.to_datetime(user_data["creation_time"][user_adoption]).apply(lambda x: x.month).value_counts()
nonadopted_months = pd.to_datetime(user_data["creation_time"][~user_adoption]).apply(lambda x: x.month).value_counts()

In [57]:
adopted_months/(adopted_months + nonadopted_months)

1     0.140914
2     0.143469
3     0.133390
4     0.090676
5     0.050172
6     0.178359
7     0.139671
8     0.165493
9     0.153163
10    0.173160
11    0.160256
12    0.138710
Name: creation_time, dtype: float64

+ It seems that there is a lot of fluctuation between adopted users based on Org ID, which would be worth looking into based on the individal organizations.

+ There is a lot of difference in adoption based on year and month. 2013 saw nearly twice as much adoption as 2012, but then 2014 had a severe drop in adoption.

+ On a month by month basis, there appear to be trends based on when a user created their account. The most severe is between May and June, where the lowest and highest values are seperated by only a month. 

+ Overall 71.2% of users are adopted, according to the analysis. Investigating reasons for the retention factors of time and organization could lead to increased retention and appear to be the most important factors. 